<a href="https://colab.research.google.com/github/pranoy234/Brain-Tumor-classification-and-detection/blob/main/Movie_recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import nltk
from nltk.stem.porter import PorterStemmer

In [47]:
dataset=pd.read_csv("/content/drive/MyDrive/movies.csv")
dataset

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [5]:
dataset.shape

(4803, 24)

In [6]:
dataset=dataset[["genres","id","keywords","tagline","original_title","overview","cast","director"]]
#for features in dataset:
#   dataset[features]=dataset[features]
#dataset
dataset.head(5)

,genres,id,keywords,tagline,original_title,overview,cast,director
0,Action Adventure Fantasy Science Fiction,19995,culture clash future space war space colony so...,Enter the World of Pandora.,Avatar,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,285,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,206647,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Spectre,A cryptic message from Bond’s past sends him o...,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,49026,dc comics crime fighter terrorist secret ident...,The Legend Ends,The Dark Knight Rises,Following the death of District Attorney Harve...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,49529,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",John Carter,"John Carter is a war-weary, former military ca...",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [7]:
dataset.isnull().sum()

genres             28
id                  0
keywords          412
tagline           844
original_title      0
overview            3
cast               43
director           30
dtype: int64

In [8]:
dataset.duplicated().sum()

0

In [9]:
dataset.dropna(inplace=True)

In [10]:
#dataset.iloc[0].crew

In [11]:
#def convert(obj):
   #L=[]
  # for i in ast.literal_eval(obj):
  #       L.append(i["name"])
  # return L

In [12]:
#dataset["crew"]=dataset["crew"].apply(convert)
#dataset
dataset["director"]=dataset["director"].apply(lambda x: x.replace(" ",""))

In [13]:
dataset["genres"]=dataset["genres"].apply(lambda x:x.split())
dataset["tagline"]=dataset["tagline"].apply(lambda x:x.split())
#dataset["original_title"]=dataset["original_title"].apply(lambda x:x.split())

dataset["overview"]=dataset["overview"].apply(lambda x:x.split())
dataset["keywords"]=dataset["keywords"].apply(lambda x:x.split())
dataset["cast"]=dataset["cast"].apply(lambda x:x.split())
dataset["director"]=dataset["director"].apply(lambda x:x.split())

In [14]:
dataset["keywords"][5]

['dual',
 'identity',
 'amnesia',
 'sandstorm',
 'love',
 'of',
 "one's",
 'life',
 'forgiveness']

In [15]:

dataset.head(2)

,genres,id,keywords,tagline,original_title,overview,cast,director
0,"[Action, Adventure, Fantasy, Science, Fiction]",19995,"[culture, clash, future, space, war, space, co...","[Enter, the, World, of, Pandora.]",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam, Worthington, Zoe, Saldana, Sigourney, We...",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug, abuse, exotic, island, east, ind...","[At, the, end, of, the, world,, the, adventure...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny, Depp, Orlando, Bloom, Keira, Knightle...",[GoreVerbinski]


In [16]:
dataset["tags"]=dataset["genres"]+dataset["keywords"]+dataset["tagline"]+dataset["overview"]+dataset["cast"]+dataset["director"]

In [17]:
dataset=dataset[["id","original_title","tags"]]


In [18]:
dataset

,id,original_title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science, Fiction,..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drug, abus..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, based, on, nov..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dc, comics, c..."
4,49529,John Carter,"[Action, Adventure, Science, Fiction, based, o..."
...,...,...,...
4792,36095,キュア,"[Crime, Horror, Mystery, Thriller, japan, pros..."
4793,182291,On The Downlow,"[Drama, confession, hazing, gang, member, lati..."
4795,124606,Bang,"[Drama, gang, audition, police, fake, homeless..."
4796,14337,Primer,"[Science, Fiction, Drama, Thriller, distrust, ..."


In [19]:
dataset["tags"]=dataset["tags"].apply(lambda x: " ".join(x))
dataset

<ipython-input-19-fb85fa773e29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["tags"]=dataset["tags"].apply(lambda x: " ".join(x))


,id,original_title,tags
0,19995,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,206647,Spectre,Action Adventure Crime spy based on novel secr...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,49529,John Carter,Action Adventure Science Fiction based on nove...
...,...,...,...
4792,36095,キュア,Crime Horror Mystery Thriller japan prostitute...
4793,182291,On The Downlow,Drama confession hazing gang member latino lgb...
4795,124606,Bang,Drama gang audition police fake homeless actre...
4796,14337,Primer,Science Fiction Drama Thriller distrust garage...


In [20]:
ps=PorterStemmer()
def stem(text):
   y=[]
   for i in text.split():
        y.append(ps.stem(i))
   return " ".join(y)

In [21]:
dataset["tags"].apply(stem)

0       action adventur fantasi scienc fiction cultur ...
1       adventur fantasi action ocean drug abus exot i...
2       action adventur crime spi base on novel secret...
3       action crime drama thriller dc comic crime fig...
4       action adventur scienc fiction base on novel m...
                              ...                        
4792    crime horror mysteri thriller japan prostitut ...
4793    drama confess haze gang member latino lgbt two...
4795    drama gang audit polic fake homeless actress s...
4796    scienc fiction drama thriller distrust garag i...
4798    action crime thriller unit states\u2013mexico ...
Name: tags, Length: 3757, dtype: object

In [22]:
vectorizr=TfidfVectorizer(max_features=5000,stop_words='english')
vectors=vectorizr.fit_transform(dataset['tags']).toarray()
len(vectorizr.get_feature_names_out())

5000

In [36]:
distance = cosine_similarity(vectors)
sorted(list(enumerate(distance[0])),reverse=True,key=lambda x:x[1])
#sorted(list(enumerate(distance[0])),reverse=True,key=lambda x:x[1][1:6])

[(0, 1.0000000000000002),
 (2746, 0.18659432847198903),
 (2175, 0.18446081742992662),
 (1429, 0.17624279922630018),
 (1771, 0.17561700289274357),
 (88, 0.17060773661209033),
 (447, 0.16260615071351545),
 (230, 0.15165101055043081),
 (2627, 0.14440494338632495),
 (3529, 0.14317704312649443),
 (357, 0.1403380411390961),
 (46, 0.13927752023734868),
 (1273, 0.13859686474555571),
 (3061, 0.13672086154881574),
 (290, 0.1322618287804045),
 (143, 0.13179225600459282),
 (2010, 0.13108029912711944),
 (1271, 0.1296431570620195),
 (2402, 0.12509561909746536),
 (638, 0.12417165992181944),
 (512, 0.12357809453452778),
 (295, 0.12342230855236645),
 (419, 0.12317864227149583),
 (3109, 0.12200268523368468),
 (1632, 0.12158767049581552),
 (742, 0.12102332569140777),
 (42, 0.11957833925140662),
 (2776, 0.11905838462143688),
 (3001, 0.11788786986114122),
 (2712, 0.11679048777299776),
 (804, 0.11566404774555275),
 (1207, 0.11455765330634467),
 (2035, 0.11386989871276537),
 (3076, 0.11355774651717832),
 (11

In [43]:
def recomend(movie):
   movie_index=dataset[dataset['original_title']==movie].index[0]
   distance_n=distance[movie_index]
   movie_list=sorted(list(enumerate(distance_n)),reverse=True,key=lambda x:x[1])[1:6]

   for i in movie_list:
       print(dataset['original_title'].iloc[i[0]])


In [53]:
recomend('Avatar')

Baby Geniuses
Ladyhawke
Broken City
The 51st State
Big Hero 6
